In [10]:
# Imports
import pandas as pd
import numpy as np
from bertopic import BERTopic

In [15]:
# Load data into df
transcripts_v1 = pd.read_csv('transcripts_dataset.csv.gz', compression='gzip')
transcripts_v1.head()

transcripts_v1.shape


(39892, 6)

In [16]:
# Load data into df
transcripts_v2 = pd.read_csv('transcripts_dataset_v2.csv.gz', compression='gzip')
transcripts_v2.head()

transcripts_v2.shape

(65468, 6)

In [ ]:
# # Sample down for exploration
# sample = list(transcripts_v1['transcript'].sample(frac=0.25))

In [9]:
# Initialize BERTopic and run

topic_model = BERTopic()
topics, probs = topic_model.fit_transform(sample)

: 

: 